In [30]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
import numpy as np

In [3]:
df = pd.read_csv("~/code/KhalylDammas/neo-hazardous-classification/raw data/nearest-earth-objects(1910-2024).csv")
df.head()

,neo_id,name,absolute_magnitude,estimated_diameter_min,estimated_diameter_max,orbiting_body,relative_velocity,miss_distance,is_hazardous
0,2162117,162117 (1998 SD15),19.14,0.394962,0.883161,Earth,71745.401048,5.814362e+07,False
1,2349507,349507 (2008 QY),18.50,0.530341,1.185878,Earth,109949.757148,5.580105e+07,True
2,2455415,455415 (2003 GA),21.45,0.136319,0.304818,Earth,24865.506798,6.720689e+07,False
3,3132126,(2002 PB),20.63,0.198863,0.444672,Earth,78890.076805,3.039644e+07,False
4,3557844,(2011 DW),22.70,0.076658,0.171412,Earth,56036.519484,6.311863e+07,False


In [7]:
#Separate features and target
X = df[['absolute_magnitude','estimated_diameter_min','relative_velocity','miss_distance']]
y = df['is_hazardous']

In [10]:
# print the X columns (features)
X.head()

,absolute_magnitude,estimated_diameter_min,relative_velocity,miss_distance
0,19.14,0.394962,71745.401048,5.814362e+07
1,18.50,0.530341,109949.757148,5.580105e+07
2,21.45,0.136319,24865.506798,6.720689e+07
3,20.63,0.198863,78890.076805,3.039644e+07
4,22.70,0.076658,56036.519484,6.311863e+07


In [11]:
X.describe()

,absolute_magnitude,estimated_diameter_min,relative_velocity,miss_distance
count,338171.000000,338171.000000,338199.000000,3.381990e+05
mean,22.932525,0.157812,51060.662908,4.153535e+07
std,2.911216,0.313885,26399.238435,2.077399e+07
min,9.250000,0.000511,203.346433,6.745533e+03
25%,20.740000,0.025384,30712.031471,2.494540e+07
50%,22.800000,0.073207,47560.465474,4.332674e+07
75%,25.100000,0.189041,66673.820614,5.933961e+07
max,33.580000,37.545248,291781.106613,7.479865e+07


In [20]:
# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(270559, 4) (67640, 4) (270559,) (67640,)


### piplene multiple models

In [22]:
# Set up the Pipeline with MinMaxScaler
pipeline = Pipeline([
    ('scaler', MinMaxScaler()),
    ('classifier', LogisticRegression())  # Default classifier, will be replaced in GridSearchCV
])

In [25]:
# Define hyperparameter grids for each model
param_grid = [
    # Logistic Regression parameters
    {'classifier': [LogisticRegression()],
     'classifier__C': [0.01, 0.1, 1, 10, 100]},

    # SVM (Linear Kernel) parameters
    {'classifier': [SVC(kernel='linear')],
     'classifier__C': [0.01, 0.1, 1, 10, 100]},

    # SVM (RBF Kernel) parameters
    {'classifier': [SVC(kernel='rbf')],
     'classifier__C': [0.01, 0.1, 1, 10, 100],
     'classifier__gamma': ['scale', 0.001, 0.01, 0.1, 1]},

    # K-Nearest Neighbors parameters
    {'classifier': [KNeighborsClassifier()],
     'classifier__n_neighbors': [3, 5, 7, 9, 11]}
]

In [27]:
# Use GridSearchCV to find the best hyperparameters
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)

In [ ]:
# Fit the grid search on the training data
grid_search.fit(X_train, y_train)

In [ ]:
# Print the best parameters and the best accuracy score
print("Best parameters found by GridSearchCV:")
print(grid_search.best_params_)

In [ ]:
# Evaluate the best model on the test set
y_pred = grid_search.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Best model accuracy on the test set: {accuracy:.4f}')

### another way for piplene multiple models

In [ ]:
pipe = Pipeline([('scaled' , MinMaxScaler()),
                 ('LR' ,LogisticRegression())])
pipe.fit(X_train ,y_train)
print(accuracy_score(y_test, pipe.predict(X_test)))

In [ ]:
pipelines = []
pipelines.append(('scaledLR' , (Pipeline([('scaled' , MinMaxScaler()),('LR' ,LogisticRegression())]))))
pipelines.append(('scaledKNN' , (Pipeline([('scaled' , MinMaxScaler()),('KNN' ,KNeighborsClassifier())]))))
pipelines.append(('scaledDT' , (Pipeline([('scaled' , MinMaxScaler()),('DT' ,DecisionTreeClassifier())]))))
pipelines.append(('scaledSVC' , (Pipeline([('scaled' , MinMaxScaler()),('SVC' ,SVC())]))))
pipelines.append(('scaledMNB' , (Pipeline([('scaled' , MinMaxScaler()),('MNB' ,GaussianNB())]))))

model_name = []
results = []
for pipe ,model in pipelines:
    kfold = KFold(n_splits=10, random_state=42)
    crossv_results = cross_val_score(model , X_train ,y_train ,cv =kfold , scoring='accuracy')
    results.append(crossv_results)
    model_name.append(pipe)
    msg = "%s: %f (%f)" % (model_name, crossv_results.mean(), crossv_results.std())
    print(msg)

In [ ]:
# Compare different Algorithms
fig = plt.figure()
fig.suptitle('Algorithm Comparison')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(model_name)
plt.show()